# Import packages

In [ ]:
%load_ext autoreload
%autoreload 2

import os, sys, sys
from pathlib import Path
for p in [Path.cwd()] + list(Path.cwd().parents):
    if p.name == 'Multifirefly-Project':
        os.chdir(p)
        sys.path.insert(0, str(p / 'multiff_analysis/multiff_code/methods'))
        break
    
from data_wrangling import specific_utils, process_monkey_information, general_utils
from pattern_discovery import pattern_by_trials, pattern_by_trials, cluster_analysis, organize_patterns_and_features
from visualization.matplotlib_tools import plot_behaviors_utils
from neural_data_analysis.neural_analysis_tools.get_neural_data import neural_data_processing
from neural_data_analysis.neural_analysis_tools.visualize_neural_data import plot_neural_data, plot_modeling_result, find_tuning_curves, raster_plot, trial_aligned_tuning_curves
from neural_data_analysis.neural_analysis_tools.test_neural_response import compare_firing_rate
from neural_data_analysis.neural_analysis_tools.model_neural_data import transform_vars, neural_data_modeling, drop_high_corr_vars, drop_high_vif_vars
from neural_data_analysis.topic_based_neural_analysis.neural_vs_behavioral import prep_monkey_data, prep_target_data, neural_vs_behavioral_class
from neural_data_analysis.topic_based_neural_analysis.planning_and_neural import planning_and_neural_class, pn_utils, pn_helper_class, pn_aligned_by_seg, pn_aligned_by_event
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class
from neural_data_analysis.neural_analysis_tools.cca_methods import cca_class, cca_utils, cca_cv_utils
from neural_data_analysis.neural_analysis_tools.cca_methods.cca_plotting import cca_plotting, cca_plot_lag_vs_no_lag, cca_plot_cv
from machine_learning.ml_methods import regression_utils, regz_regression_utils, ml_methods_class, classification_utils, ml_plotting_utils, ml_methods_utils
from planning_analysis.show_planning import nxt_ff_utils, show_planning_utils
from neural_data_analysis.neural_analysis_tools.gpfa_methods import elephant_utils, fit_gpfa_utils, plot_gpfa_utils, gpfa_helper_class
from neural_data_analysis.neural_analysis_tools.align_trials import time_resolved_regression, time_resolved_gpfa_regression,plot_time_resolved_regression

import sys
import math
import gc
import subprocess
from pathlib import Path
from importlib import reload

# Third-party imports
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import rc
from scipy import linalg, interpolate
from scipy.signal import fftconvolve
from scipy.io import loadmat
from scipy import sparse
import torch
from numpy import pi

# Machine Learning imports
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.multivariate.cancorr import CanCorr

# Neuroscience specific imports
import neo
import rcca
import quantities as pq

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
print("done")

%load_ext autoreload
%autoreload 2

# retrieve data

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0416"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0321"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Schro/data_0329"

In [ ]:
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0312"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0330"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0316"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0327"
# raw_data_folder_path = "all_monkey_data/raw_monkey_data/monkey_Bruno/data_0328"

In [ ]:
reduce_y_var_lags = False
planning_data_by_point_exists_ok = True
y_data_exists_ok = False

pn = pn_aligned_by_event.PlanningAndNeuralEventAligned(raw_data_folder_path=raw_data_folder_path)
pn.prep_data_to_analyze_planning(planning_data_by_point_exists_ok=planning_data_by_point_exists_ok,
                                 add_behav_data=False,
                                 put_data_into_bins=False)
pn.load_raw_data()
pn.spikes_df = neural_data_processing.make_spikes_df(pn.raw_data_folder_path, pn.ff_caught_T_sorted, pn.monkey_information, sampling_rate=pn.sampling_rate)

In [ ]:
#event_df = pn.get_new_ff_first_or_last_time(cur_or_nxt='cur', first_or_last='first', time_limit_to_count_sighting=2)          
pn.get_new_seg_info(cur_or_nxt='cur', first_or_last='first', time_limit_to_count_sighting=2, pre_event_window=0.25, post_event_window=0.75, exists_ok=False)

In [ ]:
stop!

# raster + fr

## both

In [ ]:
# distribute data into new_segments and add rel_spike_time
pn.get_new_seg_info(cur_or_nxt='nxt', first_or_last='first', time_limit_to_count_sighting=2, pre_event_window=0.75, post_event_window=0.75, exists_ok=False)

new_seg_info = pn.new_seg_info.copy()
new_seg_info['new_seg_start_time'] = new_seg_info['event_time'] - 1
new_seg_info['new_seg_end_time'] = new_seg_info['stop_time']
aligned_spike_trains = raster_plot.prepare_aligned_spike_trains(new_seg_info, pn.spikes_df)

In [ ]:
aligned_spike_trains

In [ ]:
aligned_spike_trains, new_seg_info = raster_plot.add_relative_times([aligned_spike_trains, new_seg_info], reference_time_col='event_time')
aligned_spike_trains = raster_plot.add_scaling_info(aligned_spike_trains, scale_anchor_col='event_time',
                                                    scale_factor_upper_col='stop_time',
                                                    scale_factor_lower_col='scale_anchor')

raster_plot.plot_rasters_and_fr(
    aligned_spike_trains, new_seg_info, pn.binned_spikes_df, pn.bin_width,
    cluster_col='cluster',
    bins_per_aggregate=300, plot_mean=True,
    max_clusters_to_plot=None, max_segments_to_plot=None,
    max_time=None
)

## plot rastors only

In [ ]:
# distribute data into new_segments and add rel_spike_time
pn.get_new_seg_info(cur_or_nxt='nxt', first_or_last='first', time_limit_to_count_sighting=2, pre_event_window=0.75, post_event_window=0.75, exists_ok=False)

new_seg_info = pn.new_seg_info.copy()
new_seg_info['new_seg_start_time'] = new_seg_info['event_time'] - 1
new_seg_info['new_seg_end_time'] = new_seg_info['stop_time']
aligned_spike_trains = raster_plot.prepare_aligned_spike_trains(new_seg_info, pn.spikes_df)

In [ ]:
aligned_spike_trains, new_seg_info = raster_plot.add_relative_times([aligned_spike_trains, new_seg_info], reference_time_col='event_time')
aligned_spike_trains = raster_plot.add_scaling_info(aligned_spike_trains, scale_anchor_col='event_time',
                                                    scale_factor_upper_col='stop_time',
                                                    scale_factor_lower_col='scale_anchor')

raster_plot.plot_rasters(aligned_spike_trains, 
                         new_seg_info=new_seg_info,
                         max_clusters_to_plot=1,
                         scale_spike_times=False,
                         col_to_rearrange_segments='rel_stop_time',
                         xmin=-0.5,
                         events_to_plot=[
                                        #'rel_event_time', 
                                         #'rel_stop_time', 
                                        #'rel_prev_ff_caught_time',
                                        #'rel_new_seg_start_time', 
                                        'rel_new_seg_end_time'
                                        ])

## plot fr only

In [ ]:
fr_df = plot_neural_data.prepare_fr_data(
        pn.binned_spikes_df, pn.bin_width, bins_per_aggregate=0, max_time=pn.ff_caught_T_new[-1])
fr_df.head()

## overall fr

In [ ]:
# check num_total_spikes / total_duration
time = pn.binned_spikes_df['bin'].max() * pn.bin_width
pn.binned_spikes_df.sum(axis=0)/ time

In [ ]:
# histplot of num_total_spikes / total_duration
time = pn.binned_spikes_df['bin'].max() * pn.bin_width
rates = pn.binned_spikes_df.sum(axis=0)/ time
rates = rates.iloc[:-1]
sns.histplot(rates.values, bins=50)

# compare fr

In [ ]:
# distribute data into new_segments and add rel_spike_time
pn.get_new_seg_info(cur_or_nxt='nxt', first_or_last='first', time_limit_to_count_sighting=2, pre_event_window=0.75, post_event_window=0.75, exists_ok=False)

In [ ]:
time_before_event = 0.9
time_after_event = 0.9
n_bins = 6

new_seg_info = pn.new_seg_info.copy()
new_seg_info['new_seg_start_time'] = new_seg_info['event_time'] - time_before_event
new_seg_info['new_seg_end_time'] = new_seg_info['event_time'] + time_after_event
aligned_spike_trains = raster_plot.prepare_aligned_spike_trains(new_seg_info, pn.spikes_df)

aligned_spike_trains, new_seg_info = raster_plot.add_relative_times([aligned_spike_trains, new_seg_info], reference_time_col='event_time')

In [ ]:
result_df = compare_firing_rate.compare_baseline_vs_post_event_from_spikes(
    aligned_spike_trains,
    pre_window=(-0.3, 0.0),
    post_window=(0.0, 0.3),
    method='wilcoxon'
)
result_df.head(10)

In [ ]:
clusters = [11, 17, 19, 21, 50, 54]
result_df[result_df['cluster_id'].isin(clusters)]